In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3


engine = sqlite3.connect('example.db')
def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, engine)

In [2]:
rsq("""-- Second query

-- Add the new column to the select statement
SELECT PlayerName, 
       Team, 
       Position, 
       AvgRebounds -- Add the new column
FROM
     -- Sub-query starts here                             
	(SELECT 
      PlayerName, 
      Team, 
      Position,
      -- Calculate average total rebounds
     (OffRebound+DefRebound)/CAST(GamesPlayed AS numeric) AS AvgRebounds
	 FROM PlayerStats) as tr
WHERE AvgRebounds >= 12; -- Filter rows""")

,PlayerName,Team,Position,AvgRebounds
0,Dwight Howard,CHO,C,12
1,Andre Drummond,DET,C,15
2,DeAndre Jordan,LAC,C,15
3,Karl-Anthony Towns,MIN,C,12
4,DeMarcus Cousins,NOP,C,12


In [ ]:
rsq("""SELECT PlayerName,
Country,
College,
DraftYear,
DraftNumber
FROM Players
-- WHERE UPPER(LEFT(College,5)) LIKE 'LOU%'
WHERE College LIKE 'Louisiana%'; -- Add the wildcard filter""")

,PlayerName,Country,College,DraftYear,DraftNumber
0,Antonio Blakeney,USA,Louisiana State,NaN,NaN
1,Ben Simmons,Australia,Louisiana State,2016.0,1.0
2,Elfrid Payton,USA,Louisiana-Lafayette,2014.0,10.0
3,Erik McCree,USA,Louisiana Tech,NaN,NaN
4,Garrett Temple,USA,Louisiana State,NaN,NaN
5,Jarell Martin,USA,Louisiana State,2015.0,25.0
6,Johnny O'Bryant,USA,Louisiana State,2014.0,36.0
7,Jordan Mickey,USA,Louisiana State,2015.0,33.0
8,Paul Millsap,USA,Louisiana Tech,2006.0,47.0
9,Tim Quarterman,USA,Louisiana State,NaN,NaN


In [10]:
rsq("""SELECT Country, COUNT(*) CountOfPlayers
FROM Players
GROUP BY Country
HAVING Country --- not good practice
IN ('Argentina','Brazil','Dominican Republic'
,'Puerto Rico');""")

,Country,CountOfPlayers
0,Argentina,2
1,Brazil,5
2,Dominican Republic,3
3,Puerto Rico,2


In [11]:
rsq("""SELECT Country, COUNT(*) CountOfPlayers
FROM Players
-- Add the filter condition
WHERE Country
-- Fill in the missing countries
IN ('Argentina','Brazil','Dominican Republic'
,'Puerto Rico')
GROUP BY Country;""")

,Country,CountOfPlayers
0,Argentina,2
1,Brazil,5
2,Dominican Republic,3
3,Puerto Rico,2


In [6]:
rsq("""SELECT Team,
SUM(TotalPoints) AS TotalPFPoints
FROM PlayerStats
-- Filter for only rows with power forwards
WHERE Position = 'PF'
GROUP BY Team
-- Filter for total points greater than 3000
HAVING SUM(TotalPoints) > 3000;""")

,Team,TotalPFPoints
0,CLE,3097


In [12]:
rsq("""SELECT Team,
SUM(TotalPoints) AS TotalCPoints
FROM PlayerStats
WHERE Position = 'C'
GROUP BY Team
HAVING SUM(TotalPoints) > 2500;""")

,Team,TotalCPoints
0,LAL,2755
1,SAS,2740


In [ ]:
SELECT TOP 25 PERCENT -- Limit rows to the upper quartile
Latitude,
Longitude,
Magnitude,
Depth,
NearestPop
FROM Earthquakes
WHERE Country = 'PG'
OR Country = 'ID'
ORDER BY Magnitude DESC; -- Order the results

In [4]:
rsq("""SELECT Latitude,
Latitude,
Longitude,
Magnitude,
Depth,
NearestPop
FROM Earthquakes
WHERE Country = 'PG'
OR Country = 'ID'
ORDER BY Magnitude DESC -- Order the results
LIMIT (SELECT COUNT(*) FROM Earthquakes) / 4;""")


,Latitude,Latitude,Longitude,Magnitude,Depth,NearestPop
0,-4.4380,-4.4380,101.367,8.40,34.00,Bengkulu
1,-0.8910,-0.8910,136.952,8.09,33.00,Biak
2,-3.9800,-3.9800,152.169,8.00,33.00,Rabaul
3,-4.5049,-4.5049,153.522,7.90,94.54,Kokopo
4,-2.6250,-2.6250,100.841,7.90,35.00,Sungai Penuh
...,...,...,...,...,...,...
138,-0.5780,-0.5780,133.130,6.50,33.00,Manokwari
139,-4.0290,-4.0290,128.020,6.50,33.00,Ambon
140,-4.2180,-4.2180,152.725,6.50,31.00,Kokopo
141,-4.8270,-4.8270,153.226,6.50,33.00,Kokopo


In [6]:
rsq("""SELECT NearestPop,
Country,
COUNT(NearestPop) NumEarthquakes -- Number of cities
FROM Earthquakes
WHERE Magnitude >= 8
AND Country IS NOT NULL
GROUP BY NearestPop, Country -- Group columns
ORDER BY NumEarthquakes DESC;""")

,NearestPop,Country,NumEarthquakes
0,Bengkulu,ID,1
1,Biak,ID,1
2,Gizo,SB,1
3,Illapel,CL,1
4,Iquique,CL,1
5,Kushiro,JP,1
6,Lata,SB,1
7,Navarro,AR,1
8,Okhotsk,RU,1
9,Quirihue,CL,1
